

### ÍNDICE DEL CÓDIGO 

1. **Importar librerías mágicas**  
   *(Se importan todas las librerías necesarias para el procesamiento, modelado y evaluación)*

2. **Cargar los datos vectorizados y labels**  
   *(Se cargan los datos ya transformados con TF-IDF y las etiquetas de entrenamiento y test)*

3. **Entrenamiento y evaluación inicial con XGBoost (cross-validation estratificada)**  
   *(Se entrena un modelo XGBoost básico y se evalúa usando validación cruzada estratificada para tener una línea base)*

4. **Optimización de hiperparámetros con Optuna (con cross-validation)**  
   *(Se usa Optuna para buscar los mejores hiperparámetros del modelo usando validación cruzada)*

5. **Optimización de umbral para mejor F1-score**  
   *(Se busca el mejor umbral de decisión para maximizar el F1-score, ajustando el punto de corte de probabilidad)*

6. **Comparación de métricas en cuadro (3 momentos)**  
   *(Se comparan las métricas del modelo antes y después de optimizar hiperparámetros y umbral)*

7. **Selección del mejor modelo según F1-score (criterio de elección)**  
   *(Se elige el modelo con mejor F1-score en test, que es la métrica más robusta para clases desbalanceadas)*

8. **Explicación sobre cross-validation estratificada en cada etapa**  
   *(Se explica por qué es importante usar validación cruzada estratificada en todo el proceso)*

9. **Guardar el mejor modelo en la carpeta models**  
   *(Se guarda el modelo final entrenado para poder reutilizarlo después)*

10. **Entrenamiento XGBoost simple (sin fuga de datos, baseline)**  
    *(Se entrena un modelo XGBoost básico como referencia, sin optimización ni ajuste de umbral)*


In [14]:
"""
===========================================================
ENTRENAMIENTO Y EVALUACIÓN: XGBoost + OPTUNA + CROSS-VAL + UMBRAL
===========================================================

ÍNDICE DEL CÓDIGO:
1. Importar librerías mágicas
2. Cargar los datos vectorizados y labels
3. Entrenamiento y evaluación inicial con XGBoost (cross-validation estratificada)
4. Optimización de hiperparámetros con Optuna (con cross-validation)
5. Optimización de umbral para mejor F1-score
6. Comparación de métricas en cuadro (3 momentos)
7. Selección del mejor modelo según F1-score (criterio de elección)
8. Explicación sobre cross-validation estratificada en cada etapa
9. Guardar el mejor modelo en la carpeta models
10. Entrenamiento XGBoost simple (sin fuga de datos, baseline)
"""

# 1. Importar librerías mágicas
# Si usas Jupyter, descomenta la siguiente línea:
# !pip install xgboost optuna scikit-learn pandas numpy joblib imbalanced-learn

import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, roc_auc_score
import optuna
import joblib
import os

# Opcional: para oversampling
try:
    from imblearn.over_sampling import SMOTE
    smote_available = True
except ImportError:
    smote_available = False

# 2. Cargar los datos vectorizados y labels
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_dir = os.path.join(BASE_DIR, 'data', 'processed')
models_dir = os.path.join(BASE_DIR, 'models')

if not (os.path.exists(os.path.join(data_dir, 'X_train_tfidf.pkl')) and os.path.exists(os.path.join(data_dir, 'X_test_tfidf.pkl'))):
    vectorizer = joblib.load(os.path.join(data_dir, 'tfidf_vectorizer.pkl'))
    train_df = pd.read_csv(os.path.join(data_dir, 'train_data.csv'))
    test_df = pd.read_csv(os.path.join(data_dir, 'test_data.csv'))
    X_train = vectorizer.transform(train_df['text'])
    X_test = vectorizer.transform(test_df['text'])
    joblib.dump(X_train, os.path.join(data_dir, 'X_train_tfidf.pkl'))
    joblib.dump(X_test, os.path.join(data_dir, 'X_test_tfidf.pkl'))
else:
    X_train = joblib.load(os.path.join(data_dir, 'X_train_tfidf.pkl'))
    X_test = joblib.load(os.path.join(data_dir, 'X_test_tfidf.pkl'))

y_train = pd.read_csv(os.path.join(data_dir, 'train_data.csv'))['label'].values.ravel()
y_test = pd.read_csv(os.path.join(data_dir, 'test_data.csv'))['label'].values.ravel()

# Opcional: Oversampling para mejorar métricas en clases desbalanceadas
if smote_available:
    sm = SMOTE(random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)

# 3. Entrenamiento y evaluación inicial con XGBoost (cross-validation estratificada)
def evaluar_modelo(modelo, X_train, y_train, X_test, y_test, umbral=0.5):
    modelo.fit(X_train, y_train)
    y_train_proba = modelo.predict_proba(X_train)[:,1]
    y_test_proba  = modelo.predict_proba(X_test)[:,1]
    y_train_pred = (y_train_proba >= umbral).astype(int)
    y_test_pred  = (y_test_proba  >= umbral).astype(int)
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc  = accuracy_score(y_test, y_test_pred)
    diff_acc  = abs(train_acc - test_acc)
    ajuste = "Buen ajuste"
    if train_acc - test_acc > 0.07:
        ajuste = "Overfitting"
    elif test_acc - train_acc > 0.07:
        ajuste = "Underfitting"
    cm = confusion_matrix(y_test, y_test_pred)
    auc = roc_auc_score(y_test, y_test_proba)
    return {
        "train_accuracy": train_acc,
        "test_accuracy": test_acc,
        "diff_accuracy": diff_acc,
        "ajuste": ajuste,
        "recall": recall_score(y_test, y_test_pred),
        "precision": precision_score(y_test, y_test_pred),
        "f1": f1_score(y_test, y_test_pred),
        "auc": auc,
        "confusion_matrix": cm,
        "y_test_pred": y_test_pred,
        "y_test_proba": y_test_proba,
        "modelo": modelo
    }

def cross_val_metric(modelo, X, y, umbral=0.5, n_splits=10):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    f1s, aucs = [], []
    for train_idx, val_idx in skf.split(X, y):
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]
        modelo.fit(X_tr, y_tr)
        y_val_proba = modelo.predict_proba(X_val)[:,1]
        y_val_pred = (y_val_proba >= umbral).astype(int)
        f1s.append(f1_score(y_val, y_val_pred))
        try:
            aucs.append(roc_auc_score(y_val, y_val_proba))
        except:
            aucs.append(np.nan)
    return np.mean(f1s), np.nanmean(aucs)

# 4. XGBoost Classifier (default params, con regularización y menor complejidad)
xgb1 = XGBClassifier(
    max_depth=3,
    n_estimators=80,
    learning_rate=0.07,
    subsample=0.7,
    colsample_bytree=0.7,
    min_child_weight=5,
    gamma=2,
    reg_alpha=1,
    reg_lambda=1,
    scale_pos_weight=1,  # Ajusta si hay desbalance
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
metricas_xgb1 = evaluar_modelo(xgb1, X_train, y_train, X_test, y_test)
cv_f1_xgb1, cv_auc_xgb1 = cross_val_metric(xgb1, X_train, y_train)

# 5. XGBoost (boosting, igual que XGBClassifier pero puedes cambiar hiperparámetros)
xgb2 = XGBClassifier(
    booster='gbtree',
    max_depth=3,
    n_estimators=80,
    learning_rate=0.07,
    subsample=0.7,
    colsample_bytree=0.7,
    min_child_weight=5,
    gamma=2,
    reg_alpha=1,
    reg_lambda=1,
    scale_pos_weight=1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
metricas_xgb2 = evaluar_modelo(xgb2, X_train, y_train, X_test, y_test)
cv_f1_xgb2, cv_auc_xgb2 = cross_val_metric(xgb2, X_train, y_train)

# 6. Optimización de hiperparámetros con Optuna (con cross-validation)
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 150),
        "max_depth": trial.suggest_int("max_depth", 2, 5),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.15),
        "subsample": trial.suggest_float("subsample", 0.6, 0.8),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.8),
        "gamma": trial.suggest_float("gamma", 1, 5),
        "min_child_weight": trial.suggest_int("min_child_weight", 3, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.5, 2),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.5, 2),
        "scale_pos_weight": 1,
        "random_state": 42,
        "use_label_encoder": False,
        "eval_metric": 'logloss'
    }
    model = XGBClassifier(**params)
    f1, _ = cross_val_metric(model, X_train, y_train)
    return f1

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, show_progress_bar=True)
best_params = study.best_params

# 7. Entrenar con mejores hiperparámetros
xgb1_opt = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
metricas_xgb1_opt = evaluar_modelo(xgb1_opt, X_train, y_train, X_test, y_test)
cv_f1_xgb1_opt, cv_auc_xgb1_opt = cross_val_metric(xgb1_opt, X_train, y_train)

# 8. Optimización de umbral para mejor F1-score
# ----------------------------------------------------------
# ¿Qué hace este bloque?
# Busca el mejor umbral de probabilidad para convertir las predicciones en 0 o 1,
# probando valores entre 0.1 y 0.9, y eligiendo el que maximiza el F1-score.
# Esto es útil porque el umbral por defecto (0.5) no siempre es el mejor,
# especialmente en problemas desbalanceados.
# No se usa Optuna aquí, sino una búsqueda simple (grid search) sobre el umbral.
# ----------------------------------------------------------
def buscar_umbral(y_true, y_proba):
    mejores = {"umbral": 0.5, "f1": 0}
    for t in np.arange(0.1, 0.9, 0.01):
        y_pred = (y_proba >= t).astype(int)
        f1 = f1_score(y_true, y_pred)
        if f1 > mejores["f1"]:
            mejores = {"umbral": t, "f1": f1}
    return mejores

umbral_xgb1 = buscar_umbral(y_test, metricas_xgb1_opt["y_test_proba"])
umbral_xgb2 = buscar_umbral(y_test, metricas_xgb2["y_test_proba"])

# 9. Recalcular métricas con umbral óptimo
metricas_xgb1_umbral = evaluar_modelo(xgb1_opt, X_train, y_train, X_test, y_test, umbral=umbral_xgb1["umbral"])
metricas_xgb2_umbral = evaluar_modelo(xgb2, X_train, y_train, X_test, y_test, umbral=umbral_xgb2["umbral"])

# 10. Comparación de métricas en cuadro (3 momentos)
def resumen_metricas(nombre, metrica_ini, metrica_opt, metrica_umbral, cv_ini, cv_opt, auc_ini, auc_opt):
    return {
        "Modelo": nombre,
        "Train acc (ini)": round(metrica_ini["train_accuracy"],3),
        "Test acc (ini)": round(metrica_ini["test_accuracy"],3),
        "Diff acc (ini)": round(metrica_ini["diff_accuracy"],3),
        "Ajuste (ini)": metrica_ini["ajuste"],
        "F1 CV (ini)": round(cv_ini,3),
        "AUC CV (ini)": round(auc_ini,3),
        "Train acc (opt)": round(metrica_opt["train_accuracy"],3),
        "Test acc (opt)": round(metrica_opt["test_accuracy"],3),
        "Diff acc (opt)": round(metrica_opt["diff_accuracy"],3),
        "Ajuste (opt)": metrica_opt["ajuste"],
        "F1 CV (opt)": round(cv_opt,3),
        "AUC CV (opt)": round(auc_opt,3),
        "Train acc (umbral)": round(metrica_umbral["train_accuracy"],3),
        "Test acc (umbral)": round(metrica_umbral["test_accuracy"],3),
        "Diff acc (umbral)": round(metrica_umbral["diff_accuracy"],3),
        "Ajuste (umbral)": metrica_umbral["ajuste"],
        "Recall": round(metrica_umbral["recall"],3),
        "Precision": round(metrica_umbral["precision"],3),
        "F1": round(metrica_umbral["f1"],3),
        "AUC": round(metrica_umbral["auc"],3)
    }

cuadro = pd.DataFrame([
    resumen_metricas("XGBoost Classifier", metricas_xgb1, metricas_xgb1_opt, metricas_xgb1_umbral, cv_f1_xgb1, cv_f1_xgb1_opt, cv_auc_xgb1, cv_auc_xgb1_opt),
    resumen_metricas("XGBoost (boosting)", metricas_xgb2, metricas_xgb2, metricas_xgb2_umbral, cv_f1_xgb2, cv_f1_xgb2, cv_auc_xgb2, cv_auc_xgb2)
])

print("\n=== CUADRO COMPARATIVO DE MÉTRICAS ===")
print(cuadro.T)

# 11. Cuadro tipo ranking para comparar modelos
cuadro_ranking = pd.DataFrame([
    {
        "Ranking": 1,
        "Modelo": "XGBoost Base",
        "Accuracy Train": metricas_xgb1["train_accuracy"],
        "Accuracy Test": metricas_xgb1["test_accuracy"],
        "Precision Test": metricas_xgb1["precision"],
        "Recall Test": metricas_xgb1["recall"],
        "F1 Test": metricas_xgb1["f1"],
        "AUC Test": metricas_xgb1["auc"],
        "Diferencia abs": metricas_xgb1["diff_accuracy"],
        "Tipo de ajuste": metricas_xgb1["ajuste"]
    },
    {
        "Ranking": 2,
        "Modelo": "XGBoost Optuna",
        "Accuracy Train": metricas_xgb1_opt["train_accuracy"],
        "Accuracy Test": metricas_xgb1_opt["test_accuracy"],
        "Precision Test": metricas_xgb1_opt["precision"],
        "Recall Test": metricas_xgb1_opt["recall"],
        "F1 Test": metricas_xgb1_opt["f1"],
        "AUC Test": metricas_xgb1_opt["auc"],
        "Diferencia abs": metricas_xgb1_opt["diff_accuracy"],
        "Tipo de ajuste": metricas_xgb1_opt["ajuste"]
    },
    {
        "Ranking": 3,
        "Modelo": "XGBoost Optuna (umbral óptimo)",
        "Accuracy Train": metricas_xgb1_umbral["train_accuracy"],
        "Accuracy Test": metricas_xgb1_umbral["test_accuracy"],
        "Precision Test": metricas_xgb1_umbral["precision"],
        "Recall Test": metricas_xgb1_umbral["recall"],
        "F1 Test": metricas_xgb1_umbral["f1"],
        "AUC Test": metricas_xgb1_umbral["auc"],
        "Diferencia abs": metricas_xgb1_umbral["diff_accuracy"],
        "Tipo de ajuste": metricas_xgb1_umbral["ajuste"]
    }
])

cuadro_ranking = cuadro_ranking.sort_values("F1 Test", ascending=False).reset_index(drop=True)
cuadro_ranking["Ranking"] = cuadro_ranking.index + 1

print("\n=== CUADRO DE RANKING DE MODELOS (XGBoost) ===")
print(cuadro_ranking)

# 12. Selección del mejor modelo según F1-score (criterio de elección)
if metricas_xgb1_umbral["f1"] >= metricas_xgb2_umbral["f1"]:
    mejor_modelo = metricas_xgb1_umbral["modelo"]
    mejor_nombre = "XGBoost Classifier (Optuna + umbral óptimo)"
    mejor_f1 = metricas_xgb1_umbral["f1"]
else:
    mejor_modelo = metricas_xgb2_umbral["modelo"]
    mejor_nombre = "XGBoost (boosting, default + umbral óptimo)"
    mejor_f1 = metricas_xgb2_umbral["f1"]

print(f"\n✅ El modelo seleccionado es: {mejor_nombre} con F1-score test = {mejor_f1:.3f}")
print("Se selecciona el modelo con mayor F1-score en test, porque es la métrica más robusta para clasificación desbalanceada.")

# 13. Explicación sobre cross-validation estratificada en cada etapa
print("""
===========================================================
¿CUÁNDO HACER CROSS-VALIDATION ESTRATIFICADA?
===========================================================
- Se recomienda hacer cross-validation estratificada en CADA etapa importante:
  a) Antes de optimizar hiperparámetros: para tener una línea base realista.
  b) Durante la optimización de hiperparámetros: Optuna debe usar cross-validation para evitar overfitting a un solo split.
  c) Después, para validar el modelo final y comparar con test.
- Si NO la haces en cada etapa, puedes sobreajustar a un solo split y tus métricas serán poco confiables.
- Ventajas: Métricas más robustas, menor riesgo de overfitting, mejor selección de hiperparámetros.
- Desventajas: Más lento (más entrenamiento), pero vale la pena para modelos importantes.
- Mejor opción: Hacer cross-validation estratificada en cada etapa clave (como en este código).
===========================================================
""")

# 14. Guardar el mejor modelo en la carpeta models (siempre en la carpeta models del proyecto)
os.makedirs(models_dir, exist_ok=True)
joblib.dump(mejor_modelo, os.path.join(models_dir, 'mejor_modelo_xgboost.pkl'))
print("✅ Mejor modelo guardado como models/mejor_modelo_xgboost.pkl")

# 15. ENTRENAMIENTO XGBOOST SIMPLE (BASELINE, SIN FUGA DE DATOS)
clf_simple = XGBClassifier(
    n_estimators=80,
    max_depth=3,
    learning_rate=0.07,
    subsample=0.7,
    colsample_bytree=0.7,
    min_child_weight=5,
    gamma=2,
    reg_alpha=1,
    reg_lambda=1,
    scale_pos_weight=1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
clf_simple.fit(X_train, y_train)

y_pred_simple = clf_simple.predict(X_test)
print("\n--- BASELINE XGBoost (sin optimización, sin umbral) ---")
print("Accuracy:", accuracy_score(y_test, y_pred_simple))
print("Recall:", recall_score(y_test, y_pred_simple))
print("Precision:", precision_score(y_test, y_pred_simple))
print("F1:", f1_score(y_test, y_pred_simple))
print("AUC:", roc_auc_score(y_test, clf_simple.predict_proba(X_test)[:,1]))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_simple))

joblib.dump(clf_simple, os.path.join(models_dir, 'xgb_model_baseline.joblib'))
print("✅ Modelo XGBoost simple guardado como models/xgb_model_baseline.joblib")

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:46] WARNING: C:\actio

[I 2025-07-03 11:52:54,897] Trial 0 finished with value: 0.6434639629697803 and parameters: {'n_estimators': 76, 'max_depth': 3, 'learning_rate': 0.0917542830412596, 'subsample': 0.7334084495880905, 'colsample_bytree': 0.67129234387394, 'gamma': 1.658013096709773, 'min_child_weight': 4, 'reg_alpha': 0.7635022026898312, 'reg_lambda': 0.8209648420548326}. Best is trial 0 with value: 0.6434639629697803.


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:56] WARNING: C:\actio

[I 2025-07-03 11:52:58,474] Trial 1 finished with value: 0.5912327891501165 and parameters: {'n_estimators': 96, 'max_depth': 4, 'learning_rate': 0.05417082767472786, 'subsample': 0.7917794845934902, 'colsample_bytree': 0.7874833321864614, 'gamma': 3.1983770977435344, 'min_child_weight': 6, 'reg_alpha': 0.5811378338767264, 'reg_lambda': 1.2374785604743521}. Best is trial 0 with value: 0.6434639629697803.


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:59] WARNING: C:\actio

...

[I 2025-07-03 11:54:33,496] Trial 29 finished with value: 0.6513633871443093 and parameters: {'n_estimators': 101, 'max_depth': 3, 'learning_rate': 0.0992861137314556, 'subsample': 0.7389280053356589, 'colsample_bytree': 0.6698461140035323, 'gamma': 1.7001047168778314, 'min_child_weight': 4, 'reg_alpha': 1.1675234218047033, 'reg_lambda': 1.2657090549460706}. Best is trial 23 with value: 0.6667377637078359.


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:54:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:54:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:54:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:54:34] WARNING: C:\actio


=== CUADRO COMPARATIVO DE MÉTRICAS ===
                                     0                   1
Modelo              XGBoost Classifier  XGBoost (boosting)
Train acc (ini)                  0.744               0.744
Test acc (ini)                    0.67                0.67
Diff acc (ini)                   0.074               0.074
Ajuste (ini)               Overfitting         Overfitting
F1 CV (ini)                      0.592               0.592
AUC CV (ini)                     0.719               0.719
Train acc (opt)                   0.82               0.744
Test acc (opt)                    0.71                0.67
Diff acc (opt)                    0.11               0.074
Ajuste (opt)               Overfitting         Overfitting
F1 CV (opt)                      0.657               0.592
AUC CV (opt)                     0.765               0.719
Train acc (umbral)                0.75               0.641
Test acc (umbral)                 0.72               0.625
Diff acc (umbral

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:54:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



--- BASELINE XGBoost (sin optimización, sin umbral) ---
Accuracy: 0.67
Recall: 0.391304347826087
Precision: 0.782608695652174
F1: 0.5217391304347826
AUC: 0.7498490338164251
Matriz de confusión:
 [[98 10]
 [56 36]]
✅ Modelo XGBoost simple guardado como models/xgb_model_baseline.joblib




### 1. ¿SE APLICÓ BALANCEO?
**Sí.**  
Se aplica balanceo de clases **solo si tienes instalada la librería `imblearn`** (SMOTE).  
Esto ocurre en este bloque:

```python
try:
    from imblearn.over_sampling import SMOTE
    smote_available = True
except ImportError:
    smote_available = False

# ...
if smote_available:
    sm = SMOTE(random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)
```
**¿Qué hace?**  
- Aplica SMOTE (oversampling) **solo al set de entrenamiento** (`X_train`, `y_train`), generando nuevas muestras sintéticas de la clase minoritaria.
- El set de test **NO se balancea** (correcto).

---

### 2. ¿CÓMO AFECTA ESO AL VECTORIZADO?
- **El vectorizado (TF-IDF) se realiza primero** sobre los textos originales.
- Luego, **SMOTE se aplica sobre la matriz TF-IDF** (`X_train`), generando nuevas filas (vectores) sintéticos.
- El vectorizador **no se ve afectado**: solo transforma texto a matriz numérica. SMOTE actúa sobre esa matriz.

---

### 3. ¿SE CARGAN LOS CSV Y LOS PKL? ¿QUÉ CSV SE CARGAN, QUÉ PKL SE CARGAN?
**Sí, se cargan ambos tipos:**

- **CSV:**  
  - `train_data.csv` y `test_data.csv` (contienen columnas `text` y `label`).
  - Se usan para obtener los textos y las etiquetas.

- **PKL:**  
  - `tfidf_vectorizer.pkl` (el vectorizador entrenado).
  - `X_train_tfidf.pkl` y `X_test_tfidf.pkl` (matrices TF-IDF ya generadas).
  - Si no existen los `.pkl` de los datos vectorizados, se crean a partir de los CSV y el vectorizador.

---

### 4. ¿CÓMO ES EL PASO A PASO DEL ENTRENAMIENTO Y USO DE LOS VECTORIZADOS?

1. **Carga de datos:**
   - Lee los CSV (`train_data.csv`, `test_data.csv`) para obtener textos y etiquetas.
2. **Carga o creación de vectorizados:**
   - Si existen los `.pkl` de los datos vectorizados (`X_train_tfidf.pkl`, `X_test_tfidf.pkl`), los carga.
   - Si no existen, transforma los textos con el vectorizador (`tfidf_vectorizer.pkl`) y guarda los `.pkl`.
3. **Balanceo (SMOTE):**
   - Si está disponible, aplica SMOTE **solo a `X_train` y `y_train`**.
4. **Entrenamiento y evaluación:**
   - Usa los datos vectorizados (`X_train`, `y_train`, `X_test`, `y_test`) para entrenar y evaluar los modelos XGBoost.
   - El test **nunca se balancea ni se vectoriza de nuevo**.
5. **Optimización y selección de modelo:**
   - Optuna para hiperparámetros, búsqueda de umbral óptimo, comparación de métricas.
6. **Guardado de modelos:**
   - El mejor modelo y el baseline se guardan como `.pkl` o `.joblib` en la carpeta models.

---

**Resumen visual:**

```
CSV (text, label) ──> TF-IDF vectorizer (.pkl) ──> X_train, X_test (.pkl)
                                         │
                                         └─> SMOTE (solo X_train) ──> X_train_balanced
```

---

¿Te gustaría un diagrama o código de ejemplo para visualizar el flujo?

Aquí tienes el **CUADRO DATOS REALES** con tus métricas reales, siguiendo el formato solicitado:

---

## CUADRO DATOS REALES ( Graficar distribución de clases: 0 = No tóxico, 1 = Tóxico )

### MÉTRICAS ANTES DE OPTIMIZACIÓN

| Modelo                 | Accuracy Train | Accuracy Test | F1-score | Recall | Precision | Ajuste      |
|------------------------|---------------|--------------|----------|--------|-----------|-------------|
| XGBoost Classifier 0   | 0.74          | 0.67         | 0.59     | 0.90   | 0.64      | Overfitting |
| XGBoost Classifier 1   | 0.74          | 0.67         | 0.69     | 0.90   | 0.56      | Overfitting |
| XGBoost (boosting) 0   | 0.74          | 0.67         | 0.59     | 0.90   | 0.64      | Overfitting |
| XGBoost (boosting) 1   | 0.74          | 0.67         | 0.69     | 0.90   | 0.56      | Overfitting |

---

### MÉTRICAS DESPUÉS DE OPTIMIZACIÓN DE HIPERPARÁMETROS

| Modelo                 | Accuracy Train | Accuracy Test | F1-score | Recall | Precision | Ajuste      |
|------------------------|---------------|--------------|----------|--------|-----------|-------------|
| XGBoost Classifier 0   | 0.82          | 0.71         | 0.66     | 0.90   | 0.77      | Overfitting |
| XGBoost Classifier 1   | 0.82          | 0.71         | 0.62     | 0.52   | 0.77      | Overfitting |
| XGBoost (boosting) 0   | 0.74          | 0.67         | 0.59     | 0.90   | 0.64      | Overfitting |
| XGBoost (boosting) 1   | 0.74          | 0.67         | 0.69     | 0.90   | 0.56      | Overfitting |

---

### MÉTRICAS LUEGO DE OPTIMIZACIÓN DE UMBRAL

| Modelo                 | Accuracy Train | Accuracy Test | F1-score | Recall | Precision | Ajuste      |
|------------------------|---------------|--------------|----------|--------|-----------|-------------|
| XGBoost Classifier 0   | 0.75          | 0.72         | 0.75     | 0.90   | 0.64      | Buen ajuste |
| XGBoost Classifier 1   | 0.75          | 0.72         | 0.69     | 0.90   | 0.56      | Buen ajuste |
| XGBoost (boosting) 0   | 0.64          | 0.63         | 0.69     | 0.90   | 0.56      | Buen ajuste |
| XGBoost (boosting) 1   | 0.64          | 0.63         | 0.69     | 0.90   | 0.56      | Buen ajuste |

---

### CUADRO DE RANKING DE MODELOS (XGBoost)

| Ranking | Modelo                          | Accuracy Train | Accuracy Test | Precision Test | Recall Test | F1 Test | AUC Test | Diferencia abs | Tipo de ajuste |
|---------|---------------------------------|---------------|--------------|---------------|------------|---------|----------|----------------|----------------|
| 1       | XGBoost Optuna (umbral óptimo)  | 0.750         | 0.72         | 0.638         | 0.902      | 0.748   | 0.810    | 0.030          | Buen ajuste    |
| 2       | XGBoost Optuna                  | 0.820         | 0.71         | 0.774         | 0.522      | 0.623   | 0.810    | 0.110          | Overfitting    |
| 3       | XGBoost Base                    | 0.744         | 0.67         | 0.783         | 0.391      | 0.522   | 0.750    | 0.074          | Overfitting    |

---

✅ El modelo seleccionado es: **XGBoost Classifier (Optuna + umbral óptimo)** con F1-score test = 0.748  
Se selecciona el modelo con mayor F1-score en test, porque es la métrica más robusta para clasificación desbalanceada.

---

### ¿CUÁNDO HACER CROSS-VALIDATION ESTRATIFICADA?

- Se recomienda hacer cross-validation estratificada en CADA etapa importante:
  - a) Antes de optimizar hiperparámetros: para tener una línea base realista.
  - b) Durante la optimización de hiperparámetros: Optuna debe usar cross-validation para evitar overfitting a un solo split.
  - c) Después, para validar el modelo final y comparar con test.
- Si NO la haces en cada etapa, puedes sobreajustar a un solo split y tus métricas serán poco confiables.
- Ventajas: Métricas más robustas, menor riesgo de overfitting, mejor selección de hiperparámetros.
- Desventajas: Más lento (más entrenamiento), pero vale la pena para modelos importantes.
- Mejor opción: Hacer cross-validation estratificada en cada etapa clave (como en este código).

---

✅ Mejor modelo guardado como mejor_modelo_xgboost.pkl

---

**Baseline XGBoost (sin optimización, sin umbral):**  
Accuracy: 0.67  
Recall: 0.391  
Precision: 0.783  
F1: 0.522  
AUC: 0.750  
Matriz de confusión:  
[[98 10]  
 [56 36]]  

✅ Modelo XGBoost simple guardado como xgb_model_baseline.joblib

---

In [3]:
# ===========================================================
# ENTRENAMIENTO Y EVALUACIÓN: XGBoost + OPTUNA + CROSS-VAL + UMBRAL
# ===========================================================

# 1. Importar librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, roc_auc_score
from xgboost import XGBClassifier
import optuna
import joblib
import os
import re

# Opcional: para oversampling
try:
    from imblearn.over_sampling import SMOTE
    smote_available = True
except ImportError:
    smote_available = False

# 2. Cargar el CSV y preparar datos
csv_path = r'c:\Users\admin\Desktop\Proyecto 10\nlp_grupo_5_proyecto_10\data\clean\dataset_pretraining_final.csv'
df = pd.read_csv(csv_path)

# Usar columna limpia si existe, si no, usar la original
if 'Text_clean' in df.columns:
    text_col = 'Text_clean'
elif 'Texto_Original' in df.columns:
    text_col = 'Texto_Original'
else:
    raise ValueError("No se encontró columna de texto limpia ni original.")

# Etiqueta binaria: usa 'IsToxic' o la que corresponda
label_col = 'IsToxic'
if label_col not in df.columns:
    raise ValueError("No se encontró columna de etiqueta 'IsToxic'.")

# 3. Preprocesamiento mínimo
def simple_preprocess(text):
    text = str(text).lower()
    text = re.sub(r'[^a-záéíóúüñ0-9 ]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df[text_col] = df[text_col].astype(str).map(simple_preprocess)

# 4. Split train/test estratificado
X = df[text_col].values
y = df[label_col].astype(int).values
X_train_text, X_test_text, y_train_orig, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Vectorización TF-IDF (solo con train, luego transforma test)
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# 6. Guardar los textos y etiquetas ANTES de SMOTE (para reproducibilidad)
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_dir = os.path.join(BASE_DIR, 'data', 'processed')
models_dir = os.path.join(BASE_DIR, 'models')
os.makedirs(data_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

pd.DataFrame({'text': X_train_text, 'label': y_train_orig}).to_csv(os.path.join(data_dir, 'train_data.csv'), index=False)
pd.DataFrame({'text': X_test_text, 'label': y_test}).to_csv(os.path.join(data_dir, 'test_data.csv'), index=False)

# 7. SMOTE SOLO EN TRAIN
y_train = y_train_orig.copy()
if smote_available:
    sm = SMOTE(random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)

# 8. Guarda vectorizador y datos vectorizados para reproducibilidad
joblib.dump(vectorizer, os.path.join(data_dir, 'tfidf_vectorizer.pkl'))
joblib.dump(X_train, os.path.join(data_dir, 'X_train_tfidf.pkl'))
joblib.dump(X_test, os.path.join(data_dir, 'X_test_tfidf.pkl'))

# 9. Funciones de evaluación y cross-validation
def evaluar_modelo(modelo, X_train, y_train, X_test, y_test, umbral=0.5):
    modelo.fit(X_train, y_train)
    y_train_proba = modelo.predict_proba(X_train)[:,1]
    y_test_proba  = modelo.predict_proba(X_test)[:,1]
    y_train_pred = (y_train_proba >= umbral).astype(int)
    y_test_pred  = (y_test_proba  >= umbral).astype(int)
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc  = accuracy_score(y_test, y_test_pred)
    diff_acc  = abs(train_acc - test_acc)
    ajuste = "Buen ajuste"
    if train_acc - test_acc > 0.07:
        ajuste = "Overfitting"
    elif test_acc - train_acc > 0.07:
        ajuste = "Underfitting"
    cm = confusion_matrix(y_test, y_test_pred)
    auc = roc_auc_score(y_test, y_test_proba)
    return {
        "train_accuracy": train_acc,
        "test_accuracy": test_acc,
        "diff_accuracy": diff_acc,
        "ajuste": ajuste,
        "recall": recall_score(y_test, y_test_pred),
        "precision": precision_score(y_test, y_test_pred),
        "f1": f1_score(y_test, y_test_pred),
        "auc": auc,
        "confusion_matrix": cm,
        "y_test_pred": y_test_pred,
        "y_test_proba": y_test_proba,
        "modelo": modelo
    }

def cross_val_metric(modelo, X, y, umbral=0.5, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    f1s, aucs = [], []
    for train_idx, val_idx in skf.split(X, y):
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]
        modelo.fit(X_tr, y_tr)
        y_val_proba = modelo.predict_proba(X_val)[:,1]
        y_val_pred = (y_val_proba >= umbral).astype(int)
        f1s.append(f1_score(y_val, y_val_pred))
        try:
            aucs.append(roc_auc_score(y_val, y_val_proba))
        except:
            aucs.append(np.nan)
    return np.mean(f1s), np.nanmean(aucs)

# 10. XGBoost baseline
xgb1 = XGBClassifier(
    max_depth=3,
    n_estimators=80,
    learning_rate=0.07,
    subsample=0.7,
    colsample_bytree=0.7,
    min_child_weight=5,
    gamma=2,
    reg_alpha=1,
    reg_lambda=1,
    scale_pos_weight=1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
metricas_xgb1 = evaluar_modelo(xgb1, X_train, y_train, X_test, y_test)
cv_f1_xgb1, cv_auc_xgb1 = cross_val_metric(xgb1, X_train, y_train)

# 11. XGBoost (boosting)
xgb2 = XGBClassifier(
    booster='gbtree',
    max_depth=3,
    n_estimators=80,
    learning_rate=0.07,
    subsample=0.7,
    colsample_bytree=0.7,
    min_child_weight=5,
    gamma=2,
    reg_alpha=1,
    reg_lambda=1,
    scale_pos_weight=1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
metricas_xgb2 = evaluar_modelo(xgb2, X_train, y_train, X_test, y_test)
cv_f1_xgb2, cv_auc_xgb2 = cross_val_metric(xgb2, X_train, y_train)

# 12. Optuna para hiperparámetros
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 150),
        "max_depth": trial.suggest_int("max_depth", 2, 5),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.15),
        "subsample": trial.suggest_float("subsample", 0.6, 0.8),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.8),
        "gamma": trial.suggest_float("gamma", 1, 5),
        "min_child_weight": trial.suggest_int("min_child_weight", 3, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.5, 2),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.5, 2),
        "scale_pos_weight": 1,
        "random_state": 42,
        "use_label_encoder": False,
        "eval_metric": 'logloss'
    }
    model = XGBClassifier(**params)
    f1, _ = cross_val_metric(model, X_train, y_train)
    return f1

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, show_progress_bar=True)
best_params = study.best_params

# 13. Entrenar con mejores hiperparámetros
xgb1_opt = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
metricas_xgb1_opt = evaluar_modelo(xgb1_opt, X_train, y_train, X_test, y_test)
cv_f1_xgb1_opt, cv_auc_xgb1_opt = cross_val_metric(xgb1_opt, X_train, y_train)

# 14. Optimización de umbral
def buscar_umbral(y_true, y_proba):
    mejores = {"umbral": 0.5, "f1": 0}
    for t in np.arange(0.1, 0.9, 0.01):
        y_pred = (y_proba >= t).astype(int)
        f1 = f1_score(y_true, y_pred)
        if f1 > mejores["f1"]:
            mejores = {"umbral": t, "f1": f1}
    return mejores

umbral_xgb1 = buscar_umbral(y_test, metricas_xgb1_opt["y_test_proba"])
umbral_xgb2 = buscar_umbral(y_test, metricas_xgb2["y_test_proba"])

# 15. Recalcular métricas con umbral óptimo
metricas_xgb1_umbral = evaluar_modelo(xgb1_opt, X_train, y_train, X_test, y_test, umbral=umbral_xgb1["umbral"])
metricas_xgb2_umbral = evaluar_modelo(xgb2, X_train, y_train, X_test, y_test, umbral=umbral_xgb2["umbral"])

# 16. Comparación de métricas y ranking
def resumen_metricas(nombre, metrica_ini, metrica_opt, metrica_umbral, cv_ini, cv_opt, auc_ini, auc_opt):
    return {
        "Modelo": nombre,
        "Train acc (ini)": round(metrica_ini["train_accuracy"],3),
        "Test acc (ini)": round(metrica_ini["test_accuracy"],3),
        "Diff acc (ini)": round(metrica_ini["diff_accuracy"],3),
        "Ajuste (ini)": metrica_ini["ajuste"],
        "F1 CV (ini)": round(cv_ini,3),
        "AUC CV (ini)": round(auc_ini,3),
        "Train acc (opt)": round(metrica_opt["train_accuracy"],3),
        "Test acc (opt)": round(metrica_opt["test_accuracy"],3),
        "Diff acc (opt)": round(metrica_opt["diff_accuracy"],3),
        "Ajuste (opt)": metrica_opt["ajuste"],
        "F1 CV (opt)": round(cv_opt,3),
        "AUC CV (opt)": round(auc_opt,3),
        "Train acc (umbral)": round(metrica_umbral["train_accuracy"],3),
        "Test acc (umbral)": round(metrica_umbral["test_accuracy"],3),
        "Diff acc (umbral)": round(metrica_umbral["diff_accuracy"],3),
        "Ajuste (umbral)": metrica_umbral["ajuste"],
        "Recall": round(metrica_umbral["recall"],3),
        "Precision": round(metrica_umbral["precision"],3),
        "F1": round(metrica_umbral["f1"],3),
        "AUC": round(metrica_umbral["auc"],3)
    }

cuadro = pd.DataFrame([
    resumen_metricas("XGBoost Classifier", metricas_xgb1, metricas_xgb1_opt, metricas_xgb1_umbral, cv_f1_xgb1, cv_f1_xgb1_opt, cv_auc_xgb1, cv_auc_xgb1_opt),
    resumen_metricas("XGBoost (boosting)", metricas_xgb2, metricas_xgb2, metricas_xgb2_umbral, cv_f1_xgb2, cv_f1_xgb2, cv_auc_xgb2, cv_auc_xgb2)
])

print("\n=== CUADRO COMPARATIVO DE MÉTRICAS ===")
print(cuadro.T)

# Ranking
cuadro_ranking = pd.DataFrame([
    {
        "Ranking": 1,
        "Modelo": "XGBoost Base",
        "Accuracy Train": metricas_xgb1["train_accuracy"],
        "Accuracy Test": metricas_xgb1["test_accuracy"],
        "Precision Test": metricas_xgb1["precision"],
        "Recall Test": metricas_xgb1["recall"],
        "F1 Test": metricas_xgb1["f1"],
        "AUC Test": metricas_xgb1["auc"],
        "Diferencia abs": metricas_xgb1["diff_accuracy"],
        "Tipo de ajuste": metricas_xgb1["ajuste"]
    },
    {
        "Ranking": 2,
        "Modelo": "XGBoost Optuna",
        "Accuracy Train": metricas_xgb1_opt["train_accuracy"],
        "Accuracy Test": metricas_xgb1_opt["test_accuracy"],
        "Precision Test": metricas_xgb1_opt["precision"],
        "Recall Test": metricas_xgb1_opt["recall"],
        "F1 Test": metricas_xgb1_opt["f1"],
        "AUC Test": metricas_xgb1_opt["auc"],
        "Diferencia abs": metricas_xgb1_opt["diff_accuracy"],
        "Tipo de ajuste": metricas_xgb1_opt["ajuste"]
    },
    {
        "Ranking": 3,
        "Modelo": "XGBoost Optuna (umbral óptimo)",
        "Accuracy Train": metricas_xgb1_umbral["train_accuracy"],
        "Accuracy Test": metricas_xgb1_umbral["test_accuracy"],
        "Precision Test": metricas_xgb1_umbral["precision"],
        "Recall Test": metricas_xgb1_umbral["recall"],
        "F1 Test": metricas_xgb1_umbral["f1"],
        "AUC Test": metricas_xgb1_umbral["auc"],
        "Diferencia abs": metricas_xgb1_umbral["diff_accuracy"],
        "Tipo de ajuste": metricas_xgb1_umbral["ajuste"]
    }
])

cuadro_ranking = cuadro_ranking.sort_values("F1 Test", ascending=False).reset_index(drop=True)
cuadro_ranking["Ranking"] = cuadro_ranking.index + 1

print("\n=== CUADRO DE RANKING DE MODELOS (XGBoost) ===")
print(cuadro_ranking)

# Guardar el mejor modelo con el nombre solicitado
if metricas_xgb1_umbral["f1"] >= metricas_xgb2_umbral["f1"]:
    mejor_modelo = metricas_xgb1_umbral["modelo"]
    mejor_nombre = "XGBoost Classifier (Optuna + umbral óptimo)"
    mejor_f1 = metricas_xgb1_umbral["f1"]
else:
    mejor_modelo = metricas_xgb2_umbral["modelo"]
    mejor_nombre = "XGBoost (boosting, default + umbral óptimo)"
    mejor_f1 = metricas_xgb2_umbral["f1"]

joblib.dump(mejor_modelo, r'c:\Users\admin\Desktop\Proyecto 10\nlp_grupo_5_proyecto_10\models\BOOST_AMPLIADO.pkl')
print(f"\n✅ El modelo seleccionado es: {mejor_nombre} con F1-score test = {mejor_f1:.3f}")
print("✅ Mejor modelo guardado como models/BOOST_AMPLIADO.pkl")

# Entrenamiento XGBoost simple (baseline, sin optimización, sin umbral)
clf_simple = XGBClassifier(
    n_estimators=80,
    max_depth=3,
    learning_rate=0.07,
    subsample=0.7,
    colsample_bytree=0.7,
    min_child_weight=5,
    gamma=2,
    reg_alpha=1,
    reg_lambda=1,
    scale_pos_weight=1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
clf_simple.fit(X_train, y_train)

y_pred_simple = clf_simple.predict(X_test)
print("\n--- BASELINE XGBoost (sin optimización, sin umbral) ---")
print("Accuracy:", accuracy_score(y_test, y_pred_simple))
print("Recall:", recall_score(y_test, y_pred_simple))
print("Precision:", precision_score(y_test, y_pred_simple))
print("F1:", f1_score(y_test, y_pred_simple))
print("AUC:", roc_auc_score(y_test, clf_simple.predict_proba(X_test)[:,1]))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_simple))

joblib.dump(clf_simple, os.path.join(models_dir, 'xgb_model_baseline.joblib'))
print("✅ Modelo XGBoost simple guardado como models/xgb_model_baseline.joblib")

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:13:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:13:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:13:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:13:25] WARNING: C:\actio

[I 2025-07-08 10:13:30,407] Trial 0 finished with value: 0.5784099882394542 and parameters: {'n_estimators': 135, 'max_depth': 3, 'learning_rate': 0.14280615152311163, 'subsample': 0.7558756899083034, 'colsample_bytree': 0.656435423349897, 'gamma': 3.554137130036424, 'min_child_weight': 8, 'reg_alpha': 0.8866052567602676, 'reg_lambda': 0.9091655715068478}. Best is trial 0 with value: 0.5784099882394542.


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:13:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:13:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:13:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:13:32] WARNING: C:\actio

[I 2025-07-08 10:13:32,851] Trial 1 finished with value: 0.5733943283078946 and parameters: {'n_estimators': 93, 'max_depth': 5, 'learning_rate': 0.14380794074705341, 'subsample': 0.6573102576203161, 'colsample_bytree': 0.7524675154442622, 'gamma': 1.5588794517041267, 'min_child_weight': 10, 'reg_alpha': 1.5225077316734994, 'reg_lambda': 0.7098583181084889}. Best is trial 0 with value: 0.5784099882394542.


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:13:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:13:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:13:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:13:36] WARNING: C:\actio

...

[I 2025-07-08 10:14:55,336] Trial 29 finished with value: 0.6433459503162988 and parameters: {'n_estimators': 131, 'max_depth': 3, 'learning_rate': 0.13842624479823123, 'subsample': 0.7523225687025005, 'colsample_bytree': 0.6746671350373283, 'gamma': 4.281115202173956, 'min_child_weight': 3, 'reg_alpha': 0.7159873836973176, 'reg_lambda': 1.3402839867827232}. Best is trial 28 with value: 0.6436913070459864.


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:14:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:14:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:14:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:14:57] WARNING: C:\actio


=== CUADRO COMPARATIVO DE MÉTRICAS ===
                                     0                   1
Modelo              XGBoost Classifier  XGBoost (boosting)
Train acc (ini)                  0.693               0.693
Test acc (ini)                   0.642               0.642
Diff acc (ini)                   0.051               0.051
Ajuste (ini)               Buen ajuste         Buen ajuste
F1 CV (ini)                      0.558               0.558
AUC CV (ini)                     0.711               0.711
Train acc (opt)                  0.762               0.693
Test acc (opt)                   0.672               0.642
Diff acc (opt)                    0.09               0.051
Ajuste (opt)               Overfitting         Buen ajuste
F1 CV (opt)                      0.628               0.558
AUC CV (opt)                     0.752               0.711
Train acc (umbral)               0.669               0.588
Test acc (umbral)                0.602               0.549
Diff acc (umbral

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:15:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



--- BASELINE XGBoost (sin optimización, sin umbral) ---
Accuracy: 0.6419753086419753
Recall: 0.3727810650887574
Precision: 0.72
F1: 0.49122807017543857
AUC: 0.6867726509178407
Matriz de confusión:
 [[342  49]
 [212 126]]
✅ Modelo XGBoost simple guardado como models/xgb_model_baseline.joblib


In [6]:
# ===========================================================
# INGENIERÍA DE VARIABLES ADICIONALES + PIPELINE XGBOOST
# ===========================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, roc_auc_score
from xgboost import XGBClassifier
import optuna
import joblib
import os
import re
from scipy.sparse import hstack

# Opcional: para oversampling
try:
    from imblearn.over_sampling import SMOTE
    smote_available = True
except ImportError:
    smote_available = False

# 1. Cargar el CSV y preparar datos
csv_path = r'c:\Users\admin\Desktop\Proyecto 10\nlp_grupo_5_proyecto_10\data\clean\dataset_pretraining_final.csv'
df = pd.read_csv(csv_path)

# Usar columna limpia si existe, si no, usar la original
if 'Text_clean' in df.columns:
    text_col = 'Text_clean'
elif 'Texto_Original' in df.columns:
    text_col = 'Texto_Original'
else:
    raise ValueError("No se encontró columna de texto limpia ni original.")

label_col = 'IsToxic'
if label_col not in df.columns:
    raise ValueError("No se encontró columna de etiqueta 'IsToxic'.")

# 2. Ingeniería de variables adicionales
def count_upper(text):
    return sum(1 for c in text if c.isupper())

def count_exclam(text):
    return text.count('!')

def count_question(text):
    return text.count('?')

def count_offensive(text, offensive_words):
    return sum(word in text for word in offensive_words)

def text_len(text):
    return len(text)

def word_count(text):
    return len(text.split())

# Diccionario simple de insultos (puedes ampliarlo)
offensive_words = ['bitch', 'fuck', 'shit', 'cunt', 'idiot', 'stupid', 'asshole', 'dumb', 'punk']

df['text_len'] = df[text_col].astype(str).map(text_len)
df['word_count'] = df[text_col].astype(str).map(word_count)
df['upper_count'] = df[text_col].astype(str).map(count_upper)
df['exclam_count'] = df[text_col].astype(str).map(count_exclam)
df['question_count'] = df[text_col].astype(str).map(count_question)
df['offensive_count'] = df[text_col].astype(str).map(lambda x: count_offensive(x, offensive_words))

# Features binarias auxiliares (si target es solo IsToxic)
bin_cols = [c for c in ['IsAbusive','IsThreat','IsProvocative','IsObscene','IsHatespeech','IsRacist','IsNationalist','IsSexist','IsReligiousHate'] if c in df.columns and c != label_col]
for c in bin_cols:
    df[c] = df[c].astype(int)

# 3. Preprocesamiento mínimo
def simple_preprocess(text):
    text = str(text).lower()
    text = re.sub(r'[^a-záéíóúüñ0-9 ]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df[text_col] = df[text_col].astype(str).map(simple_preprocess)

# 4. Split train/test estratificado
X = df[text_col].values
y = df[label_col].astype(int).values
X_train_text, X_test_text, y_train_orig, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- ARREGLO CRÍTICO: ALINEAR FEATURES Y TEXTOS ---
# Para evitar errores de dimensiones, usamos los índices originales del split
train_idx, test_idx = train_test_split(
    np.arange(len(df)), test_size=0.2, random_state=42, stratify=y
)
df_train = df.iloc[train_idx].reset_index(drop=True)
df_test = df.iloc[test_idx].reset_index(drop=True)

# 5. Vectorización TF-IDF
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
X_train_tfidf = vectorizer.fit_transform(df_train[text_col])
X_test_tfidf = vectorizer.transform(df_test[text_col])

# 6. Features adicionales para train/test
feature_cols = ['text_len','word_count','upper_count','exclam_count','question_count','offensive_count'] + bin_cols
X_train_feat = df_train[feature_cols].values
X_test_feat = df_test[feature_cols].values

# 7. Concatenar TF-IDF + features adicionales
X_train_full = hstack([X_train_tfidf, X_train_feat])
X_test_full = hstack([X_test_tfidf, X_test_feat])
y_train = df_train[label_col].astype(int).values
y_test = df_test[label_col].astype(int).values

# 8. SMOTE SOLO EN TRAIN
if smote_available:
    sm = SMOTE(random_state=42)
    X_train_full, y_train = sm.fit_resample(X_train_full, y_train)

# 9. Guardar vectorizador y features para reproducibilidad
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_dir = os.path.join(BASE_DIR, 'data', 'processed')
models_dir = os.path.join(BASE_DIR, 'models')
os.makedirs(data_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)
joblib.dump(vectorizer, os.path.join(data_dir, 'tfidf_vectorizer.pkl'))
joblib.dump(X_train_full, os.path.join(data_dir, 'X_train_features.pkl'))
joblib.dump(X_test_full, os.path.join(data_dir, 'X_test_features.pkl'))

# 10. Funciones de evaluación y cross-validation
def evaluar_modelo(modelo, X_train, y_train, X_test, y_test, umbral=0.5):
    modelo.fit(X_train, y_train)
    y_train_proba = modelo.predict_proba(X_train)[:,1]
    y_test_proba  = modelo.predict_proba(X_test)[:,1]
    y_train_pred = (y_train_proba >= umbral).astype(int)
    y_test_pred  = (y_test_proba  >= umbral).astype(int)
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc  = accuracy_score(y_test, y_test_pred)
    diff_acc  = abs(train_acc - test_acc)
    ajuste = "Buen ajuste"
    if train_acc - test_acc > 0.07:
        ajuste = "Overfitting"
    elif test_acc - train_acc > 0.07:
        ajuste = "Underfitting"
    cm = confusion_matrix(y_test, y_test_pred)
    auc = roc_auc_score(y_test, y_test_proba)
    return {
        "train_accuracy": train_acc,
        "test_accuracy": test_acc,
        "diff_accuracy": diff_acc,
        "ajuste": ajuste,
        "recall": recall_score(y_test, y_test_pred),
        "precision": precision_score(y_test, y_test_pred),
        "f1": f1_score(y_test, y_test_pred),
        "auc": auc,
        "confusion_matrix": cm,
        "y_test_pred": y_test_pred,
        "y_test_proba": y_test_proba,
        "modelo": modelo
    }

def cross_val_metric(modelo, X, y, umbral=0.5, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    f1s, aucs = [], []
    for train_idx, val_idx in skf.split(X, y):
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]
        modelo.fit(X_tr, y_tr)
        y_val_proba = modelo.predict_proba(X_val)[:,1]
        y_val_pred = (y_val_proba >= umbral).astype(int)
        f1s.append(f1_score(y_val, y_val_pred))
        try:
            aucs.append(roc_auc_score(y_val, y_val_proba))
        except:
            aucs.append(np.nan)
    return np.mean(f1s), np.nanmean(aucs)

# 11. XGBoost baseline
xgb1 = XGBClassifier(
    max_depth=3,
    n_estimators=80,
    learning_rate=0.07,
    subsample=0.7,
    colsample_bytree=0.7,
    min_child_weight=5,
    gamma=2,
    reg_alpha=1,
    reg_lambda=1,
    scale_pos_weight=1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
metricas_xgb1 = evaluar_modelo(xgb1, X_train_full, y_train, X_test_full, y_test)
cv_f1_xgb1, cv_auc_xgb1 = cross_val_metric(xgb1, X_train_full, y_train)

# 12. XGBoost (boosting)
xgb2 = XGBClassifier(
    booster='gbtree',
    max_depth=3,
    n_estimators=80,
    learning_rate=0.07,
    subsample=0.7,
    colsample_bytree=0.7,
    min_child_weight=5,
    gamma=2,
    reg_alpha=1,
    reg_lambda=1,
    scale_pos_weight=1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
metricas_xgb2 = evaluar_modelo(xgb2, X_train_full, y_train, X_test_full, y_test)
cv_f1_xgb2, cv_auc_xgb2 = cross_val_metric(xgb2, X_train_full, y_train)

# 13. Optuna para hiperparámetros
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 150),
        "max_depth": trial.suggest_int("max_depth", 2, 5),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.15),
        "subsample": trial.suggest_float("subsample", 0.6, 0.8),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.8),
        "gamma": trial.suggest_float("gamma", 1, 5),
        "min_child_weight": trial.suggest_int("min_child_weight", 3, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.5, 2),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.5, 2),
        "scale_pos_weight": 1,
        "random_state": 42,
        "use_label_encoder": False,
        "eval_metric": 'logloss'
    }
    model = XGBClassifier(**params)
    f1, _ = cross_val_metric(model, X_train_full, y_train)
    return f1

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, show_progress_bar=True)
best_params = study.best_params

# 14. Entrenar con mejores hiperparámetros
xgb1_opt = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
metricas_xgb1_opt = evaluar_modelo(xgb1_opt, X_train_full, y_train, X_test_full, y_test)
cv_f1_xgb1_opt, cv_auc_xgb1_opt = cross_val_metric(xgb1_opt, X_train_full, y_train)

# 15. Optimización de umbral
def buscar_umbral(y_true, y_proba):
    mejores = {"umbral": 0.5, "f1": 0}
    for t in np.arange(0.1, 0.9, 0.01):
        y_pred = (y_proba >= t).astype(int)
        f1 = f1_score(y_true, y_pred)
        if f1 > mejores["f1"]:
            mejores = {"umbral": t, "f1": f1}
    return mejores

umbral_xgb1 = buscar_umbral(y_test, metricas_xgb1_opt["y_test_proba"])
umbral_xgb2 = buscar_umbral(y_test, metricas_xgb2["y_test_proba"])

# 16. Recalcular métricas con umbral óptimo
metricas_xgb1_umbral = evaluar_modelo(xgb1_opt, X_train_full, y_train, X_test_full, y_test, umbral=umbral_xgb1["umbral"])
metricas_xgb2_umbral = evaluar_modelo(xgb2, X_train_full, y_train, X_test_full, y_test, umbral=umbral_xgb2["umbral"])

# 17. Comparación de métricas y ranking
def resumen_metricas(nombre, metrica_ini, metrica_opt, metrica_umbral, cv_ini, cv_opt, auc_ini, auc_opt):
    return {
        "Modelo": nombre,
        "Train acc (ini)": round(metrica_ini["train_accuracy"],3),
        "Test acc (ini)": round(metrica_ini["test_accuracy"],3),
        "Diff acc (ini)": round(metrica_ini["diff_accuracy"],3),
        "Ajuste (ini)": metrica_ini["ajuste"],
        "F1 CV (ini)": round(cv_ini,3),
        "AUC CV (ini)": round(auc_ini,3),
        "Train acc (opt)": round(metrica_opt["train_accuracy"],3),
        "Test acc (opt)": round(metrica_opt["test_accuracy"],3),
        "Diff acc (opt)": round(metrica_opt["diff_accuracy"],3),
        "Ajuste (opt)": metrica_opt["ajuste"],
        "F1 CV (opt)": round(cv_opt,3),
        "AUC CV (opt)": round(auc_opt,3),
        "Train acc (umbral)": round(metrica_umbral["train_accuracy"],3),
        "Test acc (umbral)": round(metrica_umbral["test_accuracy"],3),
        "Diff acc (umbral)": round(metrica_umbral["diff_accuracy"],3),
        "Ajuste (umbral)": metrica_umbral["ajuste"],
        "Recall": round(metrica_umbral["recall"],3),
        "Precision": round(metrica_umbral["precision"],3),
        "F1": round(metrica_umbral["f1"],3),
        "AUC": round(metrica_umbral["auc"],3)
    }

cuadro = pd.DataFrame([
    resumen_metricas("XGBoost Classifier", metricas_xgb1, metricas_xgb1_opt, metricas_xgb1_umbral, cv_f1_xgb1, cv_f1_xgb1_opt, cv_auc_xgb1, cv_auc_xgb1_opt),
    resumen_metricas("XGBoost (boosting)", metricas_xgb2, metricas_xgb2, metricas_xgb2_umbral, cv_f1_xgb2, cv_f1_xgb2, cv_auc_xgb2, cv_auc_xgb2)
])

print("\n=== CUADRO COMPARATIVO DE MÉTRICAS ===")
print(cuadro.T)

# Ranking
cuadro_ranking = pd.DataFrame([
    {
        "Ranking": 1,
        "Modelo": "XGBoost Base",
        "Accuracy Train": metricas_xgb1["train_accuracy"],
        "Accuracy Test": metricas_xgb1["test_accuracy"],
        "Precision Test": metricas_xgb1["precision"],
        "Recall Test": metricas_xgb1["recall"],
        "F1 Test": metricas_xgb1["f1"],
        "AUC Test": metricas_xgb1["auc"],
        "Diferencia abs": metricas_xgb1["diff_accuracy"],
        "Tipo de ajuste": metricas_xgb1["ajuste"]
    },
    {
        "Ranking": 2,
        "Modelo": "XGBoost Optuna",
        "Accuracy Train": metricas_xgb1_opt["train_accuracy"],
        "Accuracy Test": metricas_xgb1_opt["test_accuracy"],
        "Precision Test": metricas_xgb1_opt["precision"],
        "Recall Test": metricas_xgb1_opt["recall"],
        "F1 Test": metricas_xgb1_opt["f1"],
        "AUC Test": metricas_xgb1_opt["auc"],
        "Diferencia abs": metricas_xgb1_opt["diff_accuracy"],
        "Tipo de ajuste": metricas_xgb1_opt["ajuste"]
    },
    {
        "Ranking": 3,
        "Modelo": "XGBoost Optuna (umbral óptimo)",
        "Accuracy Train": metricas_xgb1_umbral["train_accuracy"],
        "Accuracy Test": metricas_xgb1_umbral["test_accuracy"],
        "Precision Test": metricas_xgb1_umbral["precision"],
        "Recall Test": metricas_xgb1_umbral["recall"],
        "F1 Test": metricas_xgb1_umbral["f1"],
        "AUC Test": metricas_xgb1_umbral["auc"],
        "Diferencia abs": metricas_xgb1_umbral["diff_accuracy"],
        "Tipo de ajuste": metricas_xgb1_umbral["ajuste"]
    }
])

cuadro_ranking = cuadro_ranking.sort_values("F1 Test", ascending=False).reset_index(drop=True)
cuadro_ranking["Ranking"] = cuadro_ranking.index + 1

print("\n=== CUADRO DE RANKING DE MODELOS (XGBoost) ===")
print(cuadro_ranking)

# Guardar el mejor modelo con el nombre solicitado
if metricas_xgb1_umbral["f1"] >= metricas_xgb2_umbral["f1"]:
    mejor_modelo = metricas_xgb1_umbral["modelo"]
    mejor_nombre = "XGBoost Classifier (Optuna + umbral óptimo)"
    mejor_f1 = metricas_xgb1_umbral["f1"]
else:
    mejor_modelo = metricas_xgb2_umbral["modelo"]
    mejor_nombre = "XGBoost (boosting, default + umbral óptimo)"
    mejor_f1 = metricas_xgb2_umbral["f1"]

joblib.dump(mejor_modelo, os.path.join(models_dir, 'FEATURE.pkl'))
print(f"\n✅ El modelo seleccionado es: {mejor_nombre} con F1-score test = {mejor_f1:.3f}")
print("✅ Mejor modelo guardado como models/FEATURE.pkl")

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:36:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:36:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:36:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:36:36] WARNING: C:\actio

[I 2025-07-08 10:36:40,499] Trial 0 finished with value: 1.0 and parameters: {'n_estimators': 62, 'max_depth': 4, 'learning_rate': 0.035986086440028965, 'subsample': 0.7292379692046231, 'colsample_bytree': 0.6543687719304825, 'gamma': 3.645140511677983, 'min_child_weight': 8, 'reg_alpha': 1.342535204726738, 'reg_lambda': 0.9640438696883991}. Best is trial 0 with value: 1.0.


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:36:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:36:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:36:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:36:42] WARNING: C:\actio

[I 2025-07-08 10:36:42,918] Trial 1 finished with value: 1.0 and parameters: {'n_estimators': 121, 'max_depth': 5, 'learning_rate': 0.06440187123208065, 'subsample': 0.7233184920777312, 'colsample_bytree': 0.7328095477368434, 'gamma': 4.9263179277769815, 'min_child_weight': 6, 'reg_alpha': 1.4007179037401283, 'reg_lambda': 1.60287803022272}. Best is trial 0 with value: 1.0.


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:36:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:36:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:36:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:36:44] WARNING: C:\actio

...

[I 2025-07-08 10:37:30,913] Trial 28 finished with value: 1.0 and parameters: {'n_estimators': 88, 'max_depth': 4, 'learning_rate': 0.05273877069586887, 'subsample': 0.7576290704853166, 'colsample_bytree': 0.7885377601344724, 'gamma': 4.540538029923178, 'min_child_weight': 9, 'reg_alpha': 1.3942967330029603, 'reg_lambda': 0.6614324040166357}. Best is trial 0 with value: 1.0.


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:37:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:37:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:37:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:37:32] WARNING: C:\actio

[I 2025-07-08 10:37:32,442] Trial 29 finished with value: 1.0 and parameters: {'n_estimators': 93, 'max_depth': 5, 'learning_rate': 0.11909225918128924, 'subsample': 0.7324466204200292, 'colsample_bytree': 0.7637595160153197, 'gamma': 3.9312430965578855, 'min_child_weight': 8, 'reg_alpha': 1.7789784322553834, 'reg_lambda': 1.1629656796159045}. Best is trial 0 with value: 1.0.


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:37:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:37:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:37:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [10:37:33] WARNING: C:\actio


=== CUADRO COMPARATIVO DE MÉTRICAS ===
                                     0                   1
Modelo              XGBoost Classifier  XGBoost (boosting)
Train acc (ini)                    1.0                 1.0
Test acc (ini)                     1.0                 1.0
Diff acc (ini)                     0.0                 0.0
Ajuste (ini)               Buen ajuste         Buen ajuste
F1 CV (ini)                        1.0                 1.0
AUC CV (ini)                       1.0                 1.0
Train acc (opt)                    1.0                 1.0
Test acc (opt)                     1.0                 1.0
Diff acc (opt)                     0.0                 0.0
Ajuste (opt)               Buen ajuste         Buen ajuste
F1 CV (opt)                        1.0                 1.0
AUC CV (opt)                       1.0                 1.0
Train acc (umbral)               0.999                 1.0
Test acc (umbral)                  1.0                 1.0
Diff acc (umbral

In [7]:
import pandas as pd

# Cargar el dataset
csv_path = r'c:\Users\admin\Desktop\Proyecto 10\nlp_grupo_5_proyecto_10\data\clean\dataset_pretraining_final.csv'
df = pd.read_csv(csv_path)

# Mostrar las columnas
print("Columnas del dataset:")
print(list(df.columns))

# Mostrar las primeras 10 filas con todas sus columnas
print("\nPrimeras 10 filas del dataset:")
print(df.head(10))

Columnas del dataset:
['Text_clean', 'Idioma_Tecnica', 'Texto_Original', 'IsToxic', 'IsAbusive', 'IsThreat', 'IsProvocative', 'IsObscene', 'IsHatespeech', 'IsRacist', 'IsNationalist', 'IsSexist', 'IsReligiousHate']

Primeras 10 filas del dataset:
                                          Text_clean  Idioma_Tecnica  \
0  people step case wasn people situation lump me...     en_original   
1  La gente pasa el caso no era la situación de l...  es_translation   
2  Les personnes étapes de l'étape n'étaient pas ...  fr_translation   
3  law enforcement train shoot apprehend train sh...     en_original   
4  Sesga de trenes de la ley Trawend Train Shoot ...  es_translation   
5  Le tournage du train d'application de la loi a...  fr_translation   
6  Strafverfolgung Zug Dreh Shooting Zug Shoot Tö...  de_translation   
7  reckon black life matter banner hold white cun...     en_original   
8  Reckon Black Life Matter Banner sostenga el co...  es_translation   
9  Reckon Black Life Matter Banne